In [3]:
from bs4 import BeautifulSoup as bsoup
import urllib.robotparser
import requests
import time
import pandas as pd
import numpy as np
import re
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [57]:
# Scrape daily gross for a specific date to test

url = 'https://www.boxofficemojo.com/date/2021-08-08/'
page = requests.get(url)
soup = bsoup(page.text, 'lxml')

all_divs = soup.main.find_all("div", id="table")
all_tables = all_divs[0].find_all("table")
all_trs = all_tables[0].find_all("tr")

for i in range(1, len(all_trs)):
    print(f"Entry row {i} = ")
    
    all_tds = all_trs[i].find_all("td")
    a_href = all_tds[2].find('a',href=True)
    entry = []
    
    for j in range(0, len(all_tds)):
        entry.append(all_tds[j].text)
    
    # go to individual movie page to get genres
    a_href = 'http://boxofficemojo.com' + a_href['href']
    movie_page = requests.get(a_href)
    movie_soup = bsoup(movie_page.text, 'lxml')
    
    table_div = movie_soup.find('div', {'class': 'mojo-summary-values'})
    sub_divs = table_div.find_all('div')

    genres = []
    for s in sub_divs:
        if re.search('^Genres',s.text):
            genres = s.text
            genres = re.sub('Genres', '', genres)
            genres = re.sub('\s+', ' ', genres)
    entry.append(genres.split())
        
    print(entry)

Entry row 1 = 
['1', '1', 'The Suicide Squad', '$5,940,000', '-27.6%', '-', '-', '-', '$26,205,415', '3', 'Warner Bros.\n\n', 'false', 'true', ['Action', 'Adventure', 'Comedy', 'Sci-Fi']]
Entry row 2 = 
['2', '2', 'Jungle Cruise', '$4,616,034', '-30.8%', '-50%', '4,310', '$1,071', '$65,443,044', '10', 'Walt Disney Studios Motion Pictures\n\n', 'false', 'false', ['Action', 'Adventure', 'Comedy', 'Fantasy']]
Entry row 3 = 
['3', '3', 'Old', '$1,213,725', '-28.3%', '-41.4%', '3,138', '$386', '$38,498,750', '17', 'Universal Pictures\n\n', 'false', 'false', ['Drama', 'Mystery', 'Thriller']]
Entry row 4 = 
['4', '4', 'Black Widow', '$1,082,915', '-35.1%', '-42.3%', '3,100', '$349', '$174,280,036', '31', 'Walt Disney Studios Motion Pictures\n\n', 'false', 'false', ['Action', 'Adventure', 'Sci-Fi']]
Entry row 5 = 
['5', '5', 'Stillwater', '$823,955', '-32.9%', '-41.1%', '2,611', '$315', '$9,968,760', '10', 'Focus Features\n\n', 'false', 'false', ['Crime', 'Drama', 'Thriller']]
Entry row 6 = 
[

In [69]:
# Build date for URL and make requests for each day from 2015-2019

month_and_days = {'01': [x for x in range(1,32)], '02': [x for x in range(1,29)], '03':[x for x in range(1,32)],
                  '04':[x for x in range(1,31)],'05':[x for x in range(1,32)], '06':[x for x in range(1,31)], 
                 '07':[x for x in range(1,32)], '08':[x for x in range(1,32)], '09':[x for x in range(1,31)],
                 '10': [x for x in range(1,32)], '11':[x for x in range(1,31)], '12':[x for x in range(1,32)]}

leap_years = [2008, 2016, 2020]
leap_year_month_days = {'01': [x for x in range(1,32)], '02': [x for x in range(1,30)], '03':[x for x in range(1,32)],
                  '04':[x for x in range(1,31)],'05':[x for x in range(1,32)], '06':[x for x in range(1,31)], 
                 '07':[x for x in range(1,32)], '08':[x for x in range(1,32)], '09':[x for x in range(1,31)],
                 '10': [x for x in range(1,32)], '11':[x for x in range(1,31)], '12':[x for x in range(1,32)]}

all_data = []

for year in range(2015, 2020):
    # build url for each year-month-day and scrape table data
    url = 'https://www.boxofficemojo.com/date/'
    
    if year in leap_years:
        m_d = leap_year_month_days
    else:
        m_d = month_and_days
        
    y = str(year)
    
    for month in m_d:
        m = '-' + month
        for day in m_d[month]:
            if len(str(day)) == 1:
                d = '-' + '0' + str(day)
            else:
                d = '-' + str(day)
            
            date = y + m + d
            url_curr = url + date
            
            #get data
            page = requests.get(url_curr)
            soup = bsoup(page.text, 'lxml')
            
            all_divs = soup.main.find_all("div", id="table")
            
            try:
                all_tables = all_divs[0].find_all("table")
            except:
                print(date)
            
            try:
                all_trs = all_tables[0].find_all("tr")
            except:
                print(date)
            
            for i in range(1, len(all_trs)):

                all_tds = all_trs[i].find_all("td")
                a_href = all_tds[2].find('a',href=True)
                a_href = 'http://boxofficemojo.com' + a_href['href']
                entry = []

                for j in range(0, len(all_tds)):
                    entry.append(all_tds[j].text)       
                
                entry = entry[0:11]
                entry.append(a_href)
                entry.append(date)

            
                all_data.append(entry)
    print(f'{year} processed...')

2015 processed...
2016 processed...
2017 processed...
2018 processed...
2019 processed...


In [70]:
len(all_data)

95953

In [71]:
column_names = ['TD', 'YD', 'Movie', 'Daily', '%YD', '%LW', 'Theaters', 'Avg', 'To_Date', 
                'Days', 'Distributor', 'href','Date']
df = pd.DataFrame(all_data, columns=column_names)
df

,TD,YD,Movie,Daily,%YD,%LW,Theaters,Avg,To_Date,Days,Distributor,href,Date
0,1,1,The Hobbit: The Battle of the Five Armies,"$9,320,601",+55.3%,-29%,"3,875","$2,405","$198,869,927",16,Warner Bros.\n\n,http://boxofficemojo.com/release/rl2354677249/...,2015-01-01
1,2,3,Into the Woods,"$8,081,110",+52.7%,-46.4%,"2,507","$3,223","$72,143,214",8,Walt Disney Studios Motion Pictures\n\n,http://boxofficemojo.com/release/rl5932545/?re...,2015-01-01
2,3,2,Unbroken,"$6,953,045",+17.4%,-55%,"3,167","$2,195","$69,481,655",8,Universal Pictures\n\n,http://boxofficemojo.com/release/rl3212215809/...,2015-01-01
3,4,4,Night at the Museum: Secret of the Tomb,"$5,974,323",+22.5%,-16.4%,"3,914","$1,526","$75,328,905",14,Twentieth Century Fox\n\n,http://boxofficemojo.com/release/rl2087159297/...,2015-01-01
4,5,5,Annie,"$4,257,057",+42%,-6.8%,"3,197","$1,331","$61,208,977",14,Sony Pictures Entertainment (SPE)\n\n,http://boxofficemojo.com/release/rl641893889/?...,2015-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95948,50,47,Hustlers,$185,-76.8%,-22.6%,3,$61,"$104,960,643",110,STX Entertainment\n\n,http://boxofficemojo.com/release/rl3255404033/...,2019-12-31
95949,51,52,Midnight Family,$134,-45.1%,-2.2%,2,$67,"$15,537",26,1091 Pictures\n\n,http://boxofficemojo.com/release/rl2601223681/...,2019-12-31
95950,52,53,Kind Hearts and Coronets2019 Re-release,$108,-53.6%,-,1,$108,"$27,523",35,Rialto Pictures\n\n,http://boxofficemojo.com/release/rl1923056129/...,2019-12-31
95951,53,50,Sympathy for the Devil,$73,-77.8%,-24%,1,$73,"$45,927",33,-,http://boxofficemojo.com/release/rl3399975425/...,2019-12-31


In [5]:
# Get unique list of movies via the href

href_unique = list(set(df['href']))
len(href_unique)

35600

In [6]:
# Go to each movie page to scrape genre data and add to dataframe
genre_dict = {}

for h in href_unique:
    movie_page = requests.get(h)
    movie_soup = bsoup(movie_page.text, 'lxml')

    table_div = movie_soup.find('div', {'class': 'mojo-summary-values'})
    sub_divs = table_div.find_all('div')

    for s in sub_divs:
        if re.search('^Genres',s.text):
            genres = s.text
            genres = re.sub('Genres', '', genres)
            genres = re.sub('\s+', ' ', genres)
    genre_dict[h] = genres.split()

genre_dict

KeyboardInterrupt: 

In [7]:
# Current check - this took a full hour
len(genre_dict)

4058

In [30]:
# Sanity check we're getting the correct info
genre_dict

{'http://boxofficemojo.com/release/rl2717943297/?ref_=bo_da_table_35': ['Drama'],
 'http://boxofficemojo.com/release/rl777815553/?ref_=bo_da_table_8': ['Comedy',
  'Documentary'],
 'http://boxofficemojo.com/release/rl4110779905/?ref_=bo_da_table_32': ['Biography',
  'Drama',
  'History',
  'Romance'],
 'http://boxofficemojo.com/release/rl3925902849/?ref_=bo_da_table_54': ['Documentary',
  'Drama'],
 'http://boxofficemojo.com/release/rl1852212737/?ref_=bo_da_table_35': ['Drama',
  'Romance'],
 'http://boxofficemojo.com/release/rl559449601/?ref_=bo_da_table_45': ['Comedy',
  'Crime'],
 'http://boxofficemojo.com/release/rl2105378305/?ref_=bo_da_table_7': ['Action',
  'Drama',
  'Thriller',
  'War'],
 'http://boxofficemojo.com/release/rl369133057/?ref_=bo_da_table_48': ['Drama'],
 'http://boxofficemojo.com/release/rl1417446913/?ref_=bo_da_table_35': ['Action',
  'Adventure',
  'Comedy'],
 'http://boxofficemojo.com/release/rl3481241089/?ref_=bo_da_table_4': ['Horror'],
 'http://boxofficemoj

In [58]:
# Test to unpack genres and add to dataframe
genres = genre_dict['http://boxofficemojo.com/release/rl654673409/?ref_=bo_da_table_9']
genre_str = ''
for g in genres:
    genre_str += g + '|'
df.loc[df['href']=='http://boxofficemojo.com/release/rl654673409/?ref_=bo_da_table_9', 'genre'] = genre_str
df.loc[df['href']== 'http://boxofficemojo.com/release/rl654673409/?ref_=bo_da_table_9']

,TD,YD,Movie,Daily,%YD,%LW,Theaters,Avg,To_Date,Days,Distributor,href,Date,genre
81826,9,10,Hellboy,1563100,16.3,63.2,3303.0,473.0,18703695.0,9.0,Lionsgate,http://boxofficemojo.com/release/rl654673409/?...,2019-04-20,Action|Adventure|Fantasy|Horror|Sci-Fi|


In [59]:
# For each href in the dictionary, unpack genres into string and add to genre column for that film
for h in genre_dict:
    genres = genre_dict[h]
    genre_str = ''
    for g in genres:
        genre_str += g + '|'
    df.loc[df['href'] == h,'genre'] = genre_str

df.sample(50)

,TD,YD,Movie,Daily,%YD,%LW,Theaters,Avg,To_Date,Days,Distributor,href,Date,genre
31292,47,51,Miss Hokusai,2693,38.9,NaN,2.0,1346.0,31633.0,6.0,GKIDS,http://boxofficemojo.com/release/rl2524874241/...,2016-10-19,
78981,2,2,How to Train Your Dragon: The Hidden World,6656655,98.5,52.7,4042.0,1646.0,114976260.0,16.0,Universal Pictures,http://boxofficemojo.com/release/rl2606269953/...,2019-03-09,
50068,12,12,The Stray,88408,21.4,NaN,640.0,138.0,797452.0,5.0,Purdie Distribution,http://boxofficemojo.com/release/rl1912768001/...,2017-10-10,
94547,51,53,The Fighting Preacher,69,8.0,81.1,3.0,23.0,954325.0,135.0,Purdie Distribution,http://boxofficemojo.com/release/rl3725100545/...,2019-12-05,
18815,35,38,The Forest,21812,651.1,104.1,121.0,180.0,26323858.0,43.0,Focus Features,http://boxofficemojo.com/release/rl1582466561/...,2016-02-19,
68383,34,36,We the Animals,8231,32.1,63.1,42.0,195.0,295916.0,26.0,The Orchard,http://boxofficemojo.com/release/rl1241810433/...,2018-09-11,
40268,41,43,Wilson,1093,63.6,90.2,28.0,39.0,647107.0,18.0,Fox Searchlight Pictures,http://boxofficemojo.com/release/rl3682502145/...,2017-04-10,
27026,31,30,The Conjuring 2,12451,8.0,47.1,166.0,75.0,102349812.0,62.0,Warner Bros.,http://boxofficemojo.com/release/rl3813312001/...,2016-08-10,Horror|Mystery|Thriller|
22148,12,11,Miracles from Heaven,87947,26.5,47.4,1264.0,69.0,59054907.0,43.0,TriStar Pictures,http://boxofficemojo.com/release/rl4049765889/...,2016-04-27,
18098,41,43,Only Yesterday2016 Re-release,224,47.2,83.4,1.0,224.0,57244.0,35.0,GKIDS,http://boxofficemojo.com/release/rl3481044481/...,2016-02-04,


In [65]:
unique_genre =  df.genre.str.split('|').sum()
unique_genre = set(unique_genre)
unique_genre

for g in unique_genre:
    df[g] = df.genre.map( lambda x: g in x.split('|') )
    
df.sample(50)

,TD,YD,Movie,Daily,%YD,%LW,Theaters,Avg,To_Date,Days,...,Action,Romance,Biography,Sport,Sci-Fi,Drama,Fantasy,Animation,Mystery,Adventure
30417,46,43,Mr. Church,2246,9.2,80.2,49.0,45.0,674603.0,21.0,...,False,False,False,False,False,False,False,False,False,False
27529,13,13,Lights Out,723056,45.7,47.6,942.0,767.0,63839752.0,30.0,...,False,False,False,False,False,False,False,False,False,False
90472,46,46,Brian Banks,2068,34.2,47.2,47.0,44.0,4373376.0,47.0,...,False,False,True,True,False,True,False,False,False,False
34067,4,3,Allied,617238,34.7,33.6,3160.0,195.0,31069872.0,15.0,...,False,False,False,False,False,False,False,False,False,False
91741,36,36,The Goldfinch,4386,8.5,24.4,86.0,51.0,5316242.0,35.0,...,False,False,False,False,False,False,False,False,False,False
13139,14,11,The Visit,341720,197.0,53.7,1068.0,319.0,62265685.0,36.0,...,False,False,False,False,False,False,False,False,False,False
77366,11,11,A Dog's Way Home,589190,39.4,27.2,2009.0,293.0,38936733.0,31.0,...,False,False,False,False,False,False,False,False,False,False
58258,58,-,Thelma,132,NaN,NaN,1.0,132.0,146984.0,119.0,...,False,False,False,False,False,False,False,False,False,False
66118,2,2,Mamma Mia! Here We Go Again,2863700,25.9,41.1,3514.0,814.0,79857195.0,13.0,...,False,True,False,False,False,False,False,False,False,False
29933,23,25,Queen of Katwe,26778,0.1,NaN,52.0,514.0,403009.0,7.0,...,False,False,False,False,False,False,False,False,False,False


In [60]:
# Replace non-numeric values for numeric data columns
df['Daily'] = df['Daily'].str.replace('[^0-9]', '')
df['To_Date'] = df['To_Date'].str.replace('[^0-9]', '')
df['%YD'] = df['%YD'].str.replace('+', '')
df['%YD'] = df['%YD'].str.replace('-', '')
df['%YD'] = df['%YD'].str.replace('%', '')
df['%YD'] = df['%YD'].str.replace(',', '')
df['%YD'] = df['%YD'].str.replace('<0.1', '0')
df['%LW'] = df['%LW'].str.replace('+', '')
df['%LW'] = df['%LW'].str.replace('-', '')
df['%LW'] = df['%LW'].str.replace('%', '')
df['%LW'] = df['%LW'].str.replace(',', '')
df['%LW'] = df['%LW'].str.replace('<0.1', '0')
df['Theaters'] = df['Theaters'].str.replace('[^0-9]', '')
df['Avg'] = df['Avg'].str.replace('[^0-9]', '')
df['Days'] = df['Days'].str.replace('[^0-9]', '')
df['Distributor'] = df['Distributor'].str.replace('\\n', '')
df.head()

AttributeError: Can only use .str accessor with string values!

In [74]:
# Update numeric data types
df['Daily'] = pd.to_numeric(df['Daily'])
df['%YD'] = pd.to_numeric(df['%YD'])
df['%LW'] = pd.to_numeric(df['%LW'])
df['Theaters'] = pd.to_numeric(df['Theaters'])
df['Avg'] = pd.to_numeric(df['Avg'])
df['To_Date'] = pd.to_numeric(df['To_Date'])
df['Days'] = pd.to_numeric(df['Days'])
df.dtypes

TD              object
YD              object
Movie           object
Daily            int64
%YD            float64
%LW            float64
Theaters       float64
Avg            float64
To_Date        float64
Days           float64
Distributor     object
href            object
Date            object
dtype: object

In [75]:
df['Date'] = pd.to_datetime(df['Date'])

,TD,YD,Movie,Daily,%YD,%LW,Theaters,Avg,To_Date,Days,Distributor,href,Date
32043,27,27,Kubo and the Two Strings,49048,17.2,134.1,176.0,278.0,47612370.0,73.0,Focus Features,http://boxofficemojo.com/release/rl3881600513/...,2016-10-30
84267,3,3,Rocketman,7273218,21.6,NaN,3610.0,2014.0,25725722.0,3.0,Paramount Pictures,http://boxofficemojo.com/release/rl4261709313/...,2019-06-02
85386,35,30,Wild Rose,8138,60.0,NaN,4.0,2034.0,52938.0,3.0,Neon,http://boxofficemojo.com/release/rl2349303297/...,2019-06-23
11571,40,51,The Cokeville Miracle,2759,370.0,75.6,19.0,145.0,1325176.0,102.0,Excel Entertainment,http://boxofficemojo.com/release/rl1635550721/...,2015-09-14
64240,37,35,On Chesil Beach,4483,32.9,51.3,24.0,186.0,714482.0,38.0,Bleecker Street Media,http://boxofficemojo.com/release/rl3976365569/...,2018-06-24
62576,49,46,"Love, Simon",2674,21.8,0.8,32.0,83.0,40808933.0,71.0,Twentieth Century Fox,http://boxofficemojo.com/release/rl2768274945/...,2018-05-25
73760,47,45,The Sisters Brothers,746,35.2,57.0,5.0,149.0,3131465.0,76.0,Annapurna Pictures,http://boxofficemojo.com/release/rl1376224769/...,2018-12-05
45916,1,1,Dunkirk,7530839,32.7,43.1,3748.0,2009.0,101317350.0,10.0,Warner Bros.,http://boxofficemojo.com/release/rl4118644225/...,2017-07-30
2818,2,-,The Lazarus Effect,3786762,NaN,NaN,2666.0,1420.0,3786762.0,1.0,Relativity Media,http://boxofficemojo.com/release/rl3680470529/...,2015-02-27
25967,44,42,Saturday's Warrior,656,38.6,35.8,7.0,93.0,670989.0,110.0,Purdie Distribution,http://boxofficemojo.com/release/rl4252993025/...,2016-07-19


In [66]:
df.sample(20)

,TD,YD,Movie,Daily,%YD,%LW,Theaters,Avg,To_Date,Days,...,Action,Romance,Biography,Sport,Sci-Fi,Drama,Fantasy,Animation,Mystery,Adventure
10323,39,41,The Young and Prodigious T.S. Spivet,220,13.7,86.0,9.0,24.0,76878.0,19.0,...,False,False,False,False,False,False,False,False,False,False
24744,3,-,The Shallows,6880944,NaN,NaN,2962.0,2323.0,6880944.0,1.0,...,False,False,False,False,False,False,False,False,False,False
67439,54,54,"Don't Worry, He Won't Get Far on Foot",1812,21.6,72.5,19.0,95.0,1420587.0,45.0,...,False,False,True,False,False,True,False,False,False,False
14149,38,34,Ladrones,1777,80.1,76.6,39.0,45.0,3034330.0,32.0,...,False,False,False,False,False,False,False,False,False,False
91548,9,8,It Chapter Two,407616,54.6,3.6,2303.0,176.0,207454455.0,39.0,...,False,False,False,False,False,False,False,False,False,False
82039,13,15,Kalank,187336,71.2,NaN,320.0,585.0,2083538.0,7.0,...,False,False,False,False,False,False,False,False,False,False
87289,14,14,Avengers: Endgame,154435,18.3,49.7,580.0,266.0,856861364.0,96.0,...,False,False,False,False,False,False,False,False,False,False
7013,2,2,Pitch Perfect 2,6095050,32.7,44.9,3660.0,1665.0,143387880.0,16.0,...,False,False,False,False,False,False,False,False,False,False
52556,39,38,Battle of the Sexes,9835,36.7,56.8,46.0,213.0,12552907.0,59.0,...,False,False,False,False,False,False,False,False,False,False
65814,39,39,Black Panther,5292,9.9,250.7,154.0,34.0,699948596.0,160.0,...,False,False,False,False,False,False,False,False,False,False


In [67]:
df.dtypes

TD                     object
YD                     object
Movie                  object
Daily                   int64
%YD                   float64
%LW                   float64
Theaters              float64
Avg                   float64
To_Date               float64
Days                  float64
Distributor            object
href                   object
Date           datetime64[ns]
genre                  object
                         bool
Music                    bool
Short                    bool
Family                   bool
Horror                   bool
Crime                    bool
Documentary              bool
News                     bool
Musical                  bool
History                  bool
Western                  bool
War                      bool
Comedy                   bool
Thriller                 bool
Action                   bool
Romance                  bool
Biography                bool
Sport                    bool
Sci-Fi                   bool
Drama     

In [68]:
# Send data to pickle
df.to_pickle('clean-boxofficemojo-daily_gross.pkl')